In [61]:
import requests
import h5py
import os
import cv2
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import numpy as np
import pandas as pd
from glob import glob

In [62]:
HEIGHT = 256
WIDTH = 256
CHANNELS = 3
images_path = "./images/256_ObjectCategories"
h5_file_output = "./datasets/256_images.h5"

In [63]:
def export_images():
    start = dt.datetime.now()
    dtype = h5py.special_dtype(vlen=np.dtype('uint8'))
    
    if os.path.exists(h5_file_output):
        os.remove(h5_file_output)
        
    with h5py.File(h5_file_output, "w") as wf:
        directories = next(os.walk(images_path))[1]
        asciiList = [n.split(".")[1].encode("ascii", "ignore") for n in directories]
        
        with h5py.File(h5_file_output, "a") as hf:
        
            hf.create_dataset('labels', (len(asciiList), 1), 'S10', asciiList)

            for directory in directories:
                SOURCE_IMAGES = os.path.join(images_path, directory)
                images = glob(os.path.join(SOURCE_IMAGES, "*.jpg"))

                NUM_IMAGES = len(images)
                SHAPE = (HEIGHT, WIDTH, CHANNELS)

                save_images_to_h5(images, directory, hf)

In [64]:
def save_images_to_h5(images, directory, hf):
        for i, img in enumerate(images):
            image = cv2.imread(img)
            image = cv2.resize(image, (WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)

            X_set = hf.create_dataset(
                name="X" + str(i) + "_" + directory,
                data=image,
                shape=(HEIGHT, WIDTH, CHANNELS),
                maxshape=(HEIGHT, WIDTH, CHANNELS),
                compression="gzip",
                compression_opts=9
            )

In [65]:
export_images()